
# Big‑Idea Round‑Out Notebook

Self‑contained runner that:

* validates numeric identities (50‑digit precision)  
* draws a **fully‑filled or auto‑cropped prime‑sieve heat‑map** mod 137  
* writes CSV/PNG outputs and bundles them into a ZIP for journal upload  

Just **Run all**. Add or edit identities in *Cell 5*.


In [33]:

from decimal import Decimal, getcontext
import numpy as np, matplotlib.pyplot as plt, csv, zipfile, os
getcontext().prec = 60  # 60‑digit decimal precision


In [34]:

pi  = Decimal('3.14159265358979323846264338327950288419716939937510')
phi = Decimal('1.61803398874989484820458683436563811772030917980576')
alpha_inv = Decimal('137.035999084')       # CODATA 2018
sqrt2, sqrt3 = Decimal(2).sqrt(), Decimal(3).sqrt()
print('Constants initialised.'); print()


Constants initialised.



In [35]:

def delta(a: Decimal, b: Decimal) -> Decimal:
    return abs(a - b)

def check(label, lhs: Decimal, rhs: Decimal, tol=Decimal('1e-50')):
    d = delta(lhs, rhs)
    ok = d <= tol
    badge = '✓' if ok else '✗'
    print(f'{badge}  {label:<20}  Δ = {d}')
    return [label, str(lhs), str(rhs), str(d), 'PASS' if ok else 'FAIL']


In [36]:
import random
sieve  = np.ones(limit, dtype=bool)
primes = []
for p in range(2, limit):
    if sieve[p]:
        primes.append(p)
        sieve[p*p::p] = False

sample_size = 30_000
# Ensure sample_size does not exceed the number of primes
sample_size = min(sample_size, len(primes))

sample = random.sample(primes, sample_size)

In [37]:
# ── Identity tests with adjustable tolerance ─────────────────────────
# Edit the 'identities' list and the 'tol' value as you wish.

from decimal import Decimal

tol = Decimal('1e-6')          # strict exact‑equality default
# tol = Decimal('1e-6')         # uncomment for loose “near‑miss” checks

identities = [
    # label,                   lhs,                    rhs
    ('π²  vs  2πφ',            pi**2,                  2*pi*phi),
    ('φ²  vs  π − 1',          phi**2,                 pi - Decimal(1)),
    # ('your‑label',           <Decimal expr>,        <Decimal expr>),
]

rows = [['label', 'lhs', 'rhs', 'delta', 'status']]

for lab, lhs, rhs in identities:
    rows.append(check(lab, lhs, rhs, tol=tol))

with open('identity_results.csv', 'w', newline='') as f:
    csv.writer(f).writerows(rows)

print(f'\nidentity_results.csv saved  •  tolerance = {tol}')


✗  π²  vs  2πφ           Δ = 0.29680298354116101278452702660824654834997917920151736841760
✗  φ²  vs  π − 1         Δ = 0.47644133516010160974194345108613523352313978043065360007058

identity_results.csv saved  •  tolerance = 0.000001


In [38]:
# ── Prime‑sieve heat‑map (v2) ────────────────────────────────────────
import math, numpy as np, matplotlib.pyplot as plt

# ── SETTINGS ─────────────────────────────────────────────────────────
limit      = 2_000_000        # sieve upper bound (≈ 26 659 primes) – raises density
grid_mode  = 'crop'     # 'autosize'  -> smallest square fits all primes
                             # 'crop'      -> keep fixed grid but trim empty edges
                             # 'fixed'     -> always use fixed_size × fixed_size
fixed_size = 135            # only used if grid_mode == 'fixed'
modulus    = 137            # residue base that echoes α⁻¹ theme
# ─────────────────────────────────────────────────────────────────────

# ── prime sieve up to 'limit' ---------------------------------------
sieve = np.ones(limit, dtype=bool)
primes = []
for p in range(2, limit):
    if sieve[p]:
        primes.append(p)
        sieve[p*p::p] = False

# ── build the matrix -------------------------------------------------
if grid_mode == 'autosize':
    size = math.ceil(math.sqrt(len(primes)))
elif grid_mode == 'fixed':
    size = fixed_size
else:  # crop mode keeps fixed grid first
    size = fixed_size

mat = np.zeros((size, size), dtype=int)
for i, p in enumerate(primes[: size*size ]):
    mat[i // size, i % size] = p % modulus

# ── crop away all‑zero margins if requested -------------------------
if grid_mode == 'crop':
    nz_rows = np.where(mat.sum(axis=1) != 0)[0]
    nz_cols = np.where(mat.sum(axis=0) != 0)[0]
    mat = mat[nz_rows[:, None], nz_cols]

# ── draw & save ------------------------------------------------------
plt.figure(figsize=(6, 6))
plt.imshow(mat, interpolation='nearest')
plt.axis('off')
plt.title(f'Prime residues mod {modulus}  • {len(primes):,} primes  • {grid_mode}')
plt.savefig('prime_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print('prime_heatmap.png saved.')



prime_heatmap.png saved.


In [39]:
from collections import Counter
import scipy.stats as stats
counts = Counter(p % 137 for p in primes)
expected = len(primes) / 136
chi2 = sum((c - expected)**2 / expected for c in counts.values())
print(f'χ² = {chi2:.2f}  for 135 dof  → p‑value ≈ '
      f'{1 - stats.chi2.cdf(chi2, 135):.3f}')

χ² = 1116.99  for 135 dof  → p‑value ≈ 0.000


In [40]:
from collections import Counter
import pandas as pd, numpy as np

counts = Counter(p % 137 for p in primes)
exp = len(primes) / 136
df = (pd.Series(counts).sort_index()
      .rename('observed')
      .to_frame()
      .assign(expected=exp)
      .assign(z=lambda x: (x.observed - exp) / np.sqrt(exp))
      .assign(chi=lambda x: (x.observed - exp)**2 / exp))

print(df.head())                # first few classes
print('\nTop 10 over‑ and under‑represented residues')
print(df.reindex(df.z.abs().sort_values(ascending=False).index)[:10])


   observed     expected          z          chi
0         1  1095.095588 -33.062010  1093.096501
1      1108  1095.095588   0.389953     0.152063
2      1106  1095.095588   0.329516     0.108581
3      1090  1095.095588  -0.153981     0.023710
4      1114  1095.095588   0.571264     0.326343

Top 10 over‑ and under‑represented residues
     observed     expected          z          chi
0           1  1095.095588 -33.062010  1093.096501
107      1131  1095.095588   1.084980     1.177182
100      1127  1095.095588   0.964106     0.929500
114      1127  1095.095588   0.964106     0.929500
16       1066  1095.095588  -0.879227     0.773041
17       1123  1095.095588   0.843232     0.711039
32       1123  1095.095588   0.843232     0.711039
88       1068  1095.095588  -0.818790     0.670417
134      1122  1095.095588   0.813013     0.660990
79       1121  1095.095588   0.782794     0.612767


In [41]:
from datetime import datetime
with open('README.txt','w') as f:
    f.write('Big‑Idea Reproducibility Bundle\n')
    f.write('Generated: ' + datetime.utcnow().isoformat() + 'Z\n\n')
    f.write('FILES:\n- identity_results.csv\n- prime_heatmap.png\n')

with zipfile.ZipFile('big_idea_bundle.zip', 'w') as z:
    for fname in ['identity_results.csv', 'prime_heatmap.png', 'README.txt']:
        z.write(fname)
print('big_idea_bundle.zip created.  All artefacts ready!')

big_idea_bundle.zip created.  All artefacts ready!
